In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummaryX import summary
from UnarySim.sw.kernel.linear import UnaryLinear
from UnarySim.sw.bitstream.gen import RNG, SourceGen, BSGen
from UnarySim.sw.metric.metric import ProgressiveError


In [20]:
in_feature = 2
out_feature = 1
mode = "bipolar"
scaled = True
rng = "Sobol"
bias=True

fc = nn.Linear(in_feature, out_feature, bias=bias)
if mode is "unipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(256).round().div(256)
    if bias is True:
        fc.bias.data = torch.rand(1, out_feature).mul(256).round().div(256)
elif mode is "bipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(2).sub(1).mul(256).round().div(256)
    if bias is True:
        fc.bias.data = torch.rand(1, out_feature).mul(2).sub(1).mul(256).round().div(256)
ufc = UnaryLinear(in_feature, out_feature, 256, fc.weight, fc.bias, mode=mode, scaled=scaled, bias=bias)

print("weight:")
print(fc.weight.data)
print(fc.weight.size())
if bias is True:
    print("\nbias:")
    print(fc.bias)
    print(fc.bias.size())

iVec = (torch.rand(1, in_feature)*256).round()/256
oVec = fc(iVec)

print("\ninput:")
print(iVec)
print("\noutput:")
print(oVec)

print(iVec*fc.weight.data)
print(iVec.matmul(fc.weight.data))

if scaled is False:
    oVec = oVec.clamp(-1.,1.).mul(256).floor()/256
else:
    if bias is True:
        oVec = oVec.div(in_feature).mul(256).floor()/256
    else:
        oVec = oVec.div(in_feature).mul(256).floor()/256

print("\nquantized output:")
print(oVec)

Linear(in_features=2, out_features=1, bias=True)
self.buf_wght Parameter containing:
tensor([[138, 154]])
self.buf_wght_bs BSGen()
rng_wght_idx: tensor([[0, 0]])
weight:
tensor([[0.0742, 0.2031]])
torch.Size([1, 2])

bias:
Parameter containing:
tensor([[-0.4883]], requires_grad=True)
torch.Size([1, 1])

input:
tensor([[0.2227, 0.5977]])

output:
tensor([[-0.3504]], grad_fn=<AddmmBackward>)
tensor([[0.0165, 0.1214]])


RuntimeError: size mismatch, m1: [1 x 2], m2: [1 x 2] at /Users/distiller/project/conda/conda-bld/pytorch_1565272679438/work/aten/src/TH/generic/THTensorMath.cpp:752

In [16]:
iVecSource = SourceGen(iVec, bitwidth=8, mode=mode).Gen()

iVecRNG = RNG(8, 1, rng).Out()
iVecBS = BSGen(iVecSource, iVecRNG)

iVecPP = ProgressiveError(iVec, mode=mode)
oVecPP = ProgressiveError(oVec, mode=mode)

with torch.no_grad():
    idx = torch.zeros(iVecSource.size()).type(torch.long)
    for i in range(256):
        iBS = iVecBS.Gen(idx + i)
        iVecPP.Monitor(iBS)
        oVecU = ufc(iBS)

        oVecPP.Monitor(oVecU)
    print("input error: ", min(min(iVecPP.Report())), max(max(iVecPP.Report())))
    print("output error:", min(min(oVecPP.Report())), max(max(oVecPP.Report())))
    print(oVecPP.Report())
    print(oVecPP.one_cnt)
    print(oVecPP.out_pp)



input error:  tensor(0.) tensor(0.)
output error: tensor(0.1055) tensor(0.1055)
tensor([[0.1055]])
tensor([[55.]])
tensor([[0.2148]])
